# FIFA 23 - Ultimate team players

Accessing the Futbin website to collect data of Players in FIFA 23 Ultimate Team.\

### Libs
- Pandas - To load and manipulate data
- Scrapy - To scrap data from a website

In [1]:
import pandas as pd
from selenium import webdriver    
from datetime import  date
from scrapy import Selector
import time
import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
import random

# defining today to use in some cases
today = date.today()

df_players = pd.DataFrame()

# Step 1 - Iterate over pages to get players data and sabe it on dataframe
We need to instanciate a new driver every iteration to by-pass some anti-scrap blockers.

In [2]:
# variable to pagination
page_number = 1

for page in range(200):
    # Creating a webdriver instance
    driver = webdriver.Chrome(r"Users\Felipe Mayer\Downloads\chromedriver_win32")
    url = 'https://www.futbin.com/players?page=' + str(page_number)
            
    driver.get(url)
    time.sleep(random.randint(2, 5))

    src = driver.page_source
    sel = Selector(text=src)

    players = sel.xpath('.//thead/following-sibling::tbody/tr[starts-with(@class, "player_tr")]')

    for player in players:
        index = len(df_players.index)
        df_players.loc[index, 'Name'] = player.xpath('.//a[contains(@class, "player_name_players_table")]/text()').get(default='')
        df_players.loc[index, 'Team'] = player.xpath('.//span[@class="players_club_nation"]/a[1]/@data-original-title').get(default='')
        df_players.loc[index, 'Country'] = player.xpath('.//span[@class="players_club_nation"]/a[2]/@data-original-title').get(default='')
        df_players.loc[index, 'League'] = player.xpath('.//span[@class="players_club_nation"]/a[3]/@data-original-title').get(default='')
        df_players.loc[index, 'Rating'] = player.xpath('.//span[contains(@class, "rating")]/text()').get(default='')
        rating_type = player.xpath('.//span[contains(@class, "rating")]/@class').get(default='')
        s = 'ut23'
        ss = rating_type.split(s, 1)
        df_players.loc[index, 'Rare'] = rating_type.split(s, 1)[1].split()[-1]
        df_players.loc[index, 'Version_color'] = ss[1].split(df_players.loc[index, 'Rare'], 1)[0]
        df_players.loc[index, 'Position'] = player.xpath('./td[4]/div[1]/text()').get(default='')
        df_players.loc[index, 'Other_Positions'] = player.xpath('./td[4]/div[2]/text()').get(default='')
        df_players.loc[index, 'Version'] = player.xpath('./td[5]/div[1]/text()').get(default='')
        df_players.loc[index, 'Run_type'] = player.xpath('./td[5]/div[2]/a/text()').get(default='').replace('/n', ' ').strip()
        df_players.loc[index, 'Price'] = player.xpath('./td[6]/span/text()').get(default='')
        df_players.loc[index, 'Skill'] = player.xpath('./td[7]/text()').get(default='')
        df_players.loc[index, 'Weak_foot'] = player.xpath('./td[8]/text()').get(default='')
        df_players.loc[index, 'Attack_rate'] = player.xpath('./td[9]/span[1]/text()').get(default='')
        df_players.loc[index, 'Defense_rate'] = player.xpath('./td[9]/span[2]/text()').get(default='')
        df_players.loc[index, 'Pace'] = player.xpath('./td[10]/span/text()').get(default='')
        df_players.loc[index, 'Shoot'] = player.xpath('./td[11]/span/text()').get(default='')
        df_players.loc[index, 'Pass'] = player.xpath('./td[12]/span/text()').get(default='')
        df_players.loc[index, 'Drible'] = player.xpath('./td[13]/span/text()').get(default='')
        df_players.loc[index, 'Defense'] = player.xpath('./td[14]/span/text()').get(default='')
        df_players.loc[index, 'Physical'] = player.xpath('./td[15]/span/text()').get(default='')
        df_players.loc[index, 'Body_type'] = player.xpath('./td[16]/div[2]/a/text()').get(default='')
        df_players.loc[index, 'Height_cm'] = player.xpath('./td[16]/div[1]/text()').get(default='').split('|')[0].replace('cm', ' ').strip()
        weight = player.xpath('string(./td[16]/div[2])').get(default='')
        if weight != '':
            df_players.loc[index, 'Weight (kg)'] = weight.split('(')[1].replace('kg)', ' ').strip()
        df_players.loc[index, 'Popularity'] = player.xpath('./td[17]/text()').get(default='')
        df_players.loc[index, 'Base_Stats'] = player.xpath('./td[18]/text()').get(default='')
        df_players.loc[index, 'In_Game_Stats'] = player.xpath('./td[19]/text()').get(default='')
        df_players.loc[index, 'Game_version'] = 'FIFA 23'
        df_players.loc[index, 'Updated_at'] = today

    driver.close()

    page_number += 1

display(df_players)

,Name,Team,Country,League,Rating,Rare,Version_color,Position,Other_Positions,Version,...,Defense,Physical,Body_type,Height_cm,Weight (kg),Popularity,Base_Stats,In_Game_Stats,Game_version,Updated_at
0,Pelé,FUT ICONS,Brazil,Icons,98,rare,icon gold,CAM,"CF,ST",Icon,...,60,76,Unique,173,70,3456,516,2513,FIFA 23,2023-03-28
1,Lionel Messi,Paris SG,Argentina,Ligue 1,98,rare,toty gold,RW,RM,TOTY,...,40,77,Messi,169,67,5109,504,2469,FIFA 23,2023-03-28
2,Karim Benzema,Real Madrid,France,LaLiga Santander,97,rare,toty gold,CF,ST,TOTY,...,45,90,Average,185,81,4082,508,2449,FIFA 23,2023-03-28
3,Kylian Mbappé,Paris SG,France,Ligue 1,97,rare,toty gold,ST,"CF,LW",TOTY,...,44,87,Unique,182,73,3283,512,2455,FIFA 23,2023-03-28
4,Kevin De Bruyne,Manchester City,Belgium,Premier League,97,rare,toty gold,CM,CAM,TOTY,...,81,86,Unique,181,70,2094,538,2593,FIFA 23,2023-03-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Noë Dussenne,Standard Liège,Belgium,1A Pro League (BEL 1),72,rare,silver,CB,,Normal,...,74,74,High & Average,192,85,2,321,1573,FIFA 23,2023-03-28
5996,Mats Møller Dæhli,1. FC Nürnberg,Norway,Bundesliga 2 (GER 2),72,rare,silver,CAM,,Normal,...,36,62,Lean,172,70,10,379,1780,FIFA 23,2023-03-28
5997,Daniel Johnson,Preston,Jamaica,EFL Championship (ENG 2),72,rare,silver,CAM,CM,Normal,...,55,66,Lean,174,67,3,409,1956,FIFA 23,2023-03-28
5998,Diego Fagundez,Austin FC,Uruguay,Major League Soccer,72,rare,silver,LM,"RM,LW",Normal,...,39,61,Average,173,64,2,383,1836,FIFA 23,2023-03-28


# Step 2 - Save it on a CSV to be used in futher analysis

In [4]:
df_players.to_csv('players.csv', index=False)